In [1]:
import pandas as pd

In [2]:
df_titles = pd.read_csv('../data/wileyrecs.csv')

In [4]:
df_network = pd.read_csv('../data/discourse_coalitionsNIOD2017.csv', sep=';', encoding='latin1')

In [5]:
df_network.columns


Index(['Series', 'year', 'prsauthor1', 'prs_vv', 'prs_ini', 'prsauthor2',
       'prs_vv.1', 'prs_ini.1', 'prs_preface1', 'prs_vv.2', 'prs_ini.2',
       'prs_preface2', 'Unnamed: 12', 'Unnamed: 13', 'prs_intro1',
       'Unnamed: 15', 'voorvoegsel', 'initialen', 'prs_intro2', 'Unnamed: 19',
       'Unnamed: 20', 'executor_org', 'Financier', 'Client', 'prs_Editor',
       'prs_vv.3', 'Prs_ini', 'Title'],
      dtype='object')

In [6]:
df_titles

,AU,TI,PY,JO
0,NaN,The New Director Of Icem,1961.0,Migracion
1,"Perotti, Antonio",The Migration Problem And The Encyclical “Mate...,1961.0,Migracion
2,"Marsal, Juan F.",Argentina As An Immigration Country,1961.0,Migracion
3,"Dominedo, M.",How Migration Affects The Country Of Emigration,1961.0,Migracion
4,"Zubrzycki, Jerzy",Sociological Methods For The Study Of Immigran...,1961.0,Migracion
...,...,...,...,...
1141,"Düvell, Franck",Introduction,2011.0,International Migration
1142,"Jandl, Michael","Methods, Approaches and Data Sources for Estim...",2011.0,International Migration
1143,"Prescott, Hannah",The Size of the Irregular Migrant Population i...,2011.0,International Migration
1144,"Reichel, David",Measuring Irregular Migration and Population F...,2011.0,International Migration


In [7]:
import sys
!{sys.executable} -m pip install fuzzy-search



     |████████████████████████████████| 61kB 1.5MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
!{sys.executable} -m pip install --user --upgrade pip


  Using cached https://files.pythonhosted.org/packages/de/47/58b9f3e6f611dfd17fb8bd9ed3e6f93b7ee662fb85bdfee3565e8979ddf7/pip-21.0-py3-none-any.whl


In [10]:
auts = list(df_network.prsauthor1.unique())

In [15]:
titlst = list(df_titles.AU.unique())

In [22]:
ltits = [au.split(',')[0].lower() for au in titlst if au==au]

In [21]:
lauts = [a.lower() for a in auts if a==a]

In [23]:
slauts = set(lauts)
sltits = set(ltits)

In [24]:
slauts.intersection(sltits)

{'beijer',
 'bernard',
 'cnossen',
 'johnston',
 'maselli',
 'mol',
 'sassen',
 'wander',
 'wilder-okladek'}

In [26]:
df_titles['lau'] = df_titles.AU.str.lower()

In [ ]:
df.titles.